# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125257e+02     1.727475e+00
 * time: 0.09251189231872559
     1     1.112450e+01     1.077136e+00
 * time: 1.371541976928711
     2    -1.256434e+01     1.131674e+00
 * time: 1.4919648170471191
     3    -3.445535e+01     7.998524e-01
 * time: 1.6611649990081787
     4    -4.816962e+01     5.500273e-01
 * time: 1.8176817893981934
     5    -5.711592e+01     2.389875e-01
 * time: 1.9690279960632324
     6    -5.984823e+01     1.650292e-01
 * time: 2.0799639225006104
     7    -6.091403e+01     6.038219e-02
 * time: 2.1965909004211426
     8    -6.133322e+01     7.158718e-02
 * time: 2.305989980697632
     9    -6.160558e+01     5.515181e-02
 * time: 2.423914909362793
    10    -6.180588e+01     3.521546e-02
 * time: 2.5424089431762695
    11    -6.196128e+01     2.193890e-02
 * time: 2.653932809829712
    12    -6.202948e+01     1.789150e-02
 * time: 2.7748608589172363
    13    -6.209275e+01     1.436280e-02
 * time: 2.8870759010314

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671072
    AtomicLocal         -18.8557706
    AtomicNonlocal      14.8522665
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666462032
